In [ ]:
from pathlib import Path
import os
import yaml
import numpy as np
import re

In [ ]:
dataset_path = Path("/home/shared/data_raw/exp21_outside_building/")
cam_names = [d for d in os.listdir(dataset_path / "mav0") if "cam" in d]
cam_data = {}
for cam_name in cam_names:
    with open(dataset_path / "mav0" / cam_name / "sensor.yaml", "r") as f:
        raw = f.read().replace("%YAML:1.0\n", "")
        data = yaml.safe_load(raw)

    cam_data[cam_name] = {
        "T_BS": np.array(data["T_BS"]["data"]).reshape(4, 4),
        "intrinsics": data["intrinsics"],
        "dist_coeffs": data["distortion_coefficients"],
        "resolution": data["resolution"],
    }

In [ ]:
[cam_data[cam_name]["T_BS"] for cam_name in cam_names]
# Camera data from the dataset is not valid

In [ ]:
settings_path = Path("/home/guillemc/dev/repos/OpenMAVIS/Examples/Stereo-Inertial/EuRoC.yaml")

with open(settings_path, "r") as f:
    raw = f.read().replace("%YAML:1.0\n", "")
    raw = re.sub(r"!!opencv-matrix", "", raw)
    data = yaml.safe_load(raw)

T_c1_c2 = np.array(data["Stereo.T_c1_c2"]["data"]).reshape(
    4, 4
)  # Transformation matrix from right camera to left camera
T_b_c1 = np.array(data["IMU.T_b_c1"]["data"]).reshape(4, 4)  # Transformation from body-frame (imu) to left camera

print("Body to c1\n", T_b_c1)

In [ ]:
# Go from cam1 to body frame
r_c1 = np.array([0, 0, 0, 1])
r_b = T_b_c1 @ r_c1
print(r_b)

# Go from body frame to cam1
r_b = np.array([0, 0, 0, 1])
r_c1 = np.linalg.inv(T_b_c1) @ r_b
print(r_c1)

In [ ]:
# Go from body to world
T_w_b = np.array([[0, 0, 1, 0], [0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

In [ ]:
import plotly.graph_objects as go


def plot_frame(fig, Rt, name, scale):
    R, t = Rt[:3, :3], Rt[:3, 3]
    x, y, z = R.T * scale
    fig.add_trace(
        go.Scatter3d(
            x=[t[0]],
            y=[t[1]],
            z=[t[2]],
            mode="markers+text",
            marker=dict(size=3),
            text=[name],
            textposition="top center",
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=[t[0], t[0] + x[0]],
            y=[t[1], t[1] + x[1]],
            z=[t[2], t[2] + x[2]],
            mode="lines",
            line=dict(width=4, color="red"),
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=[t[0], t[0] + y[0]],
            y=[t[1], t[1] + y[1]],
            z=[t[2], t[2] + y[2]],
            mode="lines",
            line=dict(width=4, color="green"),
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=[t[0], t[0] + z[0]],
            y=[t[1], t[1] + z[1]],
            z=[t[2], t[2] + z[2]],
            mode="lines",
            line=dict(width=4, color="blue"),
        )
    )

In [ ]:
fig = go.Figure()
scale = 0.01

plot_frame(fig, T_w_b, "IMU", scale)
plot_frame(fig, T_w_b @ T_b_c1, "Cam1 (letf)", scale)
plot_frame(fig, T_w_b @ T_b_c1 @ T_c1_c2, "Cam2 (right)", scale)

fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    showlegend=False,
    width=800,
    height=400,
)
fig.show()

In [ ]:
settings_path = Path("/home/guillemc/dev/repos/OpenMAVIS/Examples/Multi-Inertial/HiltiChallenge2022.yaml")

with open(settings_path, "r") as f:
    raw = f.read().replace("%YAML:1.0\n", "")
    raw = re.sub(r"!!opencv-matrix", "", raw)
    data = yaml.safe_load(raw)

T_w_b = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
T_c1_c2 = np.array(data["Stereo.T_c1_c2"]["data"]).reshape(4, 4)
T_b_c1 = np.array(data["IMU.T_b_c1"]["data"]).reshape(4, 4)
T_b_c3 = np.array(data["IMU.T_b_c3"]["data"]).reshape(4, 4)
T_b_c4 = np.array(data["IMU.T_b_c4"]["data"]).reshape(4, 4)

In [ ]:
fig = go.Figure()
scale = 0.01

plot_frame(fig, T_w_b, "IMU", scale)
plot_frame(fig, T_w_b @ T_b_c1, "Cam1 (front-left)", scale)
plot_frame(fig, T_w_b @ T_b_c1 @ T_c1_c2, "Cam2 (front-right)", scale)
plot_frame(fig, T_w_b @ T_b_c3, "Cam3 (left)", scale)
plot_frame(fig, T_w_b @ T_b_c4, "Cam4 (right)", scale)


fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    showlegend=False,
    width=800,
    height=400,
)
fig.show()

In [ ]:
from pathlib import Path
import re
import yaml

settings_path = Path("/home/guillemc/dev/repos/OpenMAVIS/Examples/Multi-Inertial/LAC.yaml")

T_w_b = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

H = 720
W = 1280
fx = 915.7
fy = 515.08
cx = 640.0
cy = 360.0
k1 = 0.0
k2 = 0.0
k3 = 0.0
k4 = 0.0

# Forward facing stereo camera pair
T_b_c1 = np.array([[0, 0, +1, +0.28], [-1, 0, 0, +0.081], [0, -1, 0, 0.131], [0, 0, 0, 1]])
T_b_c2 = np.array([[0, 0, +1, +0.28], [-1, 0, 0, -0.081], [0, -1, 0, 0.131], [0, 0, 0, 1]])

# Mid-chassis cameras
T_b_c3 = np.array([[+1, 0, 0, +0.015], [0, 0, +1, +0.081], [0, -1, 0, 0.132], [0, 0, 0, 1]])
T_b_c4 = np.array([[-1, 0, 0, +0.015], [0, 0, -1, -0.081], [0, -1, 0, 0.132], [0, 0, 0, 1]])

# Rearward facing stereo camera pair
T_b_c5 = np.array([[0, 0, -1, -0.28], [+1, 0, 0, +0.081], [0, -1, 0, 0.131], [0, 0, 0, 1]])
T_b_c6 = np.array([[0, 0, -1, -0.28], [+1, 0, 0, -0.081], [0, -1, 0, 0.131], [0, 0, 0, 1]])

T_c1_c2 = np.linalg.inv(T_b_c1) @ T_b_c2

fig = go.Figure()
scale = 0.1

plot_frame(fig, T_w_b, "IMU", scale)
plot_frame(fig, T_w_b @ T_b_c1, "Cam1 (front-left)", scale)
# plot_frame(fig, T_w_b @ T_b_c2, "Cam2 (front-right)", scale)
plot_frame(fig, T_w_b @ T_b_c1 @ T_c1_c2, "Cam2 (front-right)", scale)
plot_frame(fig, T_w_b @ T_b_c5, "Cam5 (back-left)", scale)
plot_frame(fig, T_w_b @ T_b_c6, "Cam6 (back-right)", scale)
plot_frame(fig, T_w_b @ T_b_c3, "Cam3 (left)", scale)
plot_frame(fig, T_w_b @ T_b_c4, "Cam4 (right)", scale)

fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    showlegend=False,
    width=800,
    height=400,
)
fig.show()

In [ ]:
in_settings_path = Path("/home/guillemc/dev/repos/OpenMAVIS/Examples/Multi-Inertial/HiltiChallenge2022.yaml")
out_settings_path = Path("/home/guillemc/dev/repos/OpenMAVIS/Examples/Multi-Inertial/LAC.yaml")


def opencv_matrix_block(name, mat):
    flat = ", ".join(f"{v:.8f}" for v in mat.flatten())
    return f"""{name}: !!opencv-matrix
  rows: 4
  cols: 4
  dt: f
  data: [{flat}]
"""


with open(in_settings_path, "r") as f:
    raw = f.read().replace("%YAML:1.0\n", "")
    raw = re.sub(r"!!opencv-matrix", "", raw)
    data = yaml.safe_load(raw)

# Update intrinsics
for i in range(1, 7):
    data[f"Camera{i}.fx"] = fx
    data[f"Camera{i}.fy"] = fy
    data[f"Camera{i}.cx"] = cx
    data[f"Camera{i}.cy"] = cy
    data[f"Camera{i}.k1"] = 0.0
    data[f"Camera{i}.k2"] = 0.0
    data[f"Camera{i}.k3"] = 0.0
    data[f"Camera{i}.k4"] = 0.0
    data[f"Camera{i}.p1"] = 0.0
    data[f"Camera{i}.p2"] = 0.0
    data[f"Camera{i}.overlappingBegin"] = 0
    data[f"Camera{i}.overlappingEnd"] = W

data["Camera.tye"] = "PinHole"
data["Camera.width"] = W
data["Camera.height"] = H
data["Camera.fps"] = 10

# Default
# data["ORBextractor.iniThFAST"] = 15
# data["ORBextractor.minThFAST"] = 7
# data["ORBextractor.nFeatures"] = 500
# data["ORBextractor.nLevels"] = 8
# data["ORBextractor.scaleFactor"] = 1.2
# data["Stereo.ThDepth"] = 40.0

data["ORBextractor.iniThFAST"] = 5
data["ORBextractor.minThFAST"] = 1
data["ORBextractor.nFeatures"] = 2500
data["ORBextractor.nLevels"] = 15
data["ORBextractor.scaleFactor"] = 1.2
data["Stereo.ThDepth"] = 200.0

# IMU.AccWalk: 0.00022
# IMU.Frequency: 400.0
# IMU.GyroWalk: 3.1e-06
# IMU.NoiseAcc: 0.0086
# IMU.NoiseGyro: 0.000171
# data["IMU.AccWalk"] = 0.0000022
# data["IMU.Frequency"] = 10.0
# data["IMU.GyroWalk"] = 1.0e-8
# data["IMU.NoiseAcc"] = 0.000086
# data["IMU.NoiseGyro"] = 0.00000171

# Remove matrix fields to prevent YAML from dumping them
mat_keys = ["IMU.T_b_c1", "IMU.T_b_c2", "IMU.T_b_c3", "IMU.T_b_c4", "IMU.T_b_c5", "IMU.T_b_c6", "Stereo.T_c1_c2"]
_ = [data.pop(k, None) for k in mat_keys]

# Dump the rest of the file
yaml_text = "%YAML:1.0\n" + yaml.dump(data, default_flow_style=False)

# Append OpenCV matrix blocks
yaml_text += "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c1", T_b_c1) + "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c2", T_b_c2) + "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c3", T_b_c3) + "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c4", T_b_c4) + "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c5", T_b_c5) + "\n"
yaml_text += opencv_matrix_block("IMU.T_b_c6", T_b_c6) + "\n"
yaml_text += opencv_matrix_block("Stereo.T_c1_c2", T_c1_c2) + "\n"

# Save the updated file
with open(out_settings_path, "w") as f:
    f.write(yaml_text)

In [ ]:
import json
import csv
import os

basedir = "/home/shared/data_raw/LAC/segmentation/slam_map1_preset1_teleop/"
basedir = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0/"
with open(os.path.join(basedir, "data_log.json")) as f:
    data = json.load(f)

In [ ]:
data["frames"][0]

In [ ]:
cam_rows = []
imu_rows = []
Rt_true = []

for frame in data["frames"]:
    if frame["step"] < 80:
        continue
    if frame["step"] > 1e6:
        break
    timestamp_ns = str(int(frame["mission_time"] * 1e9))
    if frame["step"] % 2 == 0:
        cam_rows.append([timestamp_ns, f"{frame['step']:06d}"])

    imu = frame["imu"]
    row = [str(timestamp_ns)] + [str(imu[i]) for i in range(3, 6)] + [str(imu[i]) for i in range(0, 3)]
    imu_rows.append(row)

    Rt_true.append(np.array(frame["pose"]))

Rt_true = np.array(Rt_true)

with open(os.path.join(basedir, "data_cam.csv"), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["#timestamp [ns]", "step"])
    writer.writerows(cam_rows)

with open(os.path.join(basedir, "data_imu.csv"), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "#timestamp [ns]",
            "w_RS_S_x [rad s^-1]",
            "w_RS_S_y [rad s^-1]",
            "w_RS_S_z [rad s^-1]",
            "a_RS_S_x [m s^-2]",
            "a_RS_S_y [m s^-2]",
            "a_RS_S_z [m s^-2]",
        ]
    )
    writer.writerows(imu_rows)

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread(os.path.join(basedir, "FrontLeft", f"{80:06d}.png"))
plt.imshow(img)
plt.show()


def preprocess_image(img):
    img = cv2.equalizeHist(img)  # For grayscale
    img = cv2.GaussianBlur(img, (3, 3), 0)
    return img


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
processed = preprocess_image(gray)
plt.imshow(processed, cmap="gray")
plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

outdir = "/home/guillemc/dev/repos/OpenMAVIS/"

mode = "STEREO"
with open(os.path.join(outdir, f"f_{mode}.txt"), "r") as f:
    rq_f = np.loadtxt(f, skiprows=1)

with open(os.path.join(outdir, f"kf_{mode}.txt"), "r") as f:
    rq_kf = np.loadtxt(f, skiprows=1)

r_f = []
for i in range(len(rq_f)):
    r_tmp = np.concatenate((rq_f[i, 1:4], [1]))
    # T_tmp = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
    r_f.append(Rt_true[0] @ np.linalg.inv(T_w_b) @ r_tmp)
    # r_f.append(r_tmp)
r_f = np.array(r_f)

r_kf = []
for i in range(len(rq_kf)):
    r_tmp = np.concatenate((rq_kf[i, 1:4], [1]))
    r_kf.append(Rt_true[0] @ r_tmp)
r_kf = np.array(r_kf)

t_max = 5000

fig = go.Figure()

fig.add_scatter3d(
    x=r_f[:t_max, 0],
    y=r_f[:t_max, 1],
    z=r_f[:t_max, 2],
    mode="lines",
    line=dict(width=4, color="blue"),
)
# fig.add_scatter3d(
#     x=r_kf[:t_max, 0],
#     y=r_kf[:t_max, 1],
#     z=r_kf[:t_max, 2],
#     mode="lines",
#     line=dict(width=4, color="green"),
# )
fig.add_scatter3d(
    x=[Rt_true[0, 0, 3]],
    y=[Rt_true[0, 1, 3]],
    z=[Rt_true[0, 2, 3]],
    mode="markers+text",
    marker=dict(size=3),
)
fig.add_scatter3d(
    x=Rt_true[:, 0, 3],
    y=Rt_true[:, 1, 3],
    z=Rt_true[:, 2, 3],
    mode="lines",
    line=dict(width=4, color="red"),
)

fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    showlegend=False,
    width=800,
    height=400,
)
fig.show()

In [ ]:
Rt_true[0]